needs beautifulsoup4 langchain aperturedb

after adb config create local --active --no-interactive
need to edit port # to match: vi ~/Library/Application\ Support/aperturedb/adb.json

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [ ]:
files = os.listdir("../../city_data")
file_texts = []
for file in files:
    with open(f"../../city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=64, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

In [ ]:
from langchain_community.vectorstores import ApertureDB

vector_db = ApertureDB.from_documents(file_texts, embeddings)

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a chain that passes documents to an LLM
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

# Treat the vectorstore as a document retriever
retriever = vector_db.as_retriever()

# Create a RAG chain that connects the retriever to the LLM
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
user_query = "How long has Boston been a city?"
response = retrieval_chain.invoke({"input": user_query})
print(response["answer"])